# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
# from api_keys import g_key

#Enter your API key
g_key=""


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv("../WeatherPy/output/output.csv")
del df["Unnamed: 0"]
df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temp.,Wind Speed
0,Poronaysk,69,RU,1586552314,92,49.22,143.12,24.85,11.45
1,Tuktoyaktuk,40,CA,1586552316,71,69.45,-133.04,8.60,10.29
2,Santiago,0,CL,1586552272,49,-33.46,-70.65,71.73,5.82
3,Lebu,57,CL,1586552318,80,-37.62,-73.65,60.28,8.08
4,Tezu,0,IN,1586552319,33,27.92,96.17,68.14,3.62
...,...,...,...,...,...,...,...,...,...
564,Oulu,20,FI,1586553028,90,65.00,27.00,27.00,4.14
565,Laguna de Perlas,40,NI,1586553029,66,12.34,-83.67,87.80,4.70
566,Ocós,5,GT,1586553030,49,14.51,-92.19,94.06,11.41
567,Kloulklubed,75,PW,1586553033,88,7.04,134.26,77.00,4.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
location=df[["Latitude", "Longitude"]]
weight=df["Humidity"]
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=weight, 
                                 dissipating=False, max_intensity=82,
                                 point_radius=2)


# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
#Enter desired weather conditions
max_temp=80
min_temp=70
wind_speed=10
cloudiness=0

In [10]:
newdf=df.loc[(df["Maximum Temp."]<max_temp) &(df["Maximum Temp."] >min_temp)]
newdf=newdf.loc[newdf["Wind Speed"]<wind_speed]
newdf=newdf.loc[newdf["Cloudiness"]==cloudiness]
newdf=newdf.reset_index()
del newdf["index"]
newdf

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temp.,Wind Speed
0,Santiago,0,CL,1586552272,49,-33.46,-70.65,71.73,5.82
1,Gossaigaon,0,IN,1586552328,40,26.45,89.99,72.43,5.14
2,Gwadar,0,PK,1586552465,68,25.12,62.33,79.54,3.62
3,Pisco,0,PE,1586552460,30,-13.70,-76.22,73.11,9.17
4,Benjamín Hill,0,MX,1586552496,18,30.17,-111.10,77.07,8.99
5,Port Hedland,0,AU,1586552707,88,-20.32,118.57,71.60,5.82
6,Chodavaram,0,IN,1586552740,63,17.83,82.95,79.48,4.00
7,Marzuq,0,YE,1586552763,42,14.40,46.47,71.37,3.31
8,Tura,0,IN,1586552795,36,25.52,90.22,71.89,3.87
9,Pozo Colorado,0,PY,1586552825,36,-23.49,-58.80,77.00,4.09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df=newdf
hotel_df["Hotel Name"]=""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temp.,Wind Speed,Hotel Name
0,Santiago,0,CL,1586552272,49,-33.46,-70.65,71.73,5.82,
1,Gossaigaon,0,IN,1586552328,40,26.45,89.99,72.43,5.14,
2,Gwadar,0,PK,1586552465,68,25.12,62.33,79.54,3.62,
3,Pisco,0,PE,1586552460,30,-13.70,-76.22,73.11,9.17,
4,Benjamín Hill,0,MX,1586552496,18,30.17,-111.10,77.07,8.99,
5,Port Hedland,0,AU,1586552707,88,-20.32,118.57,71.60,5.82,
6,Chodavaram,0,IN,1586552740,63,17.83,82.95,79.48,4.00,
7,Marzuq,0,YE,1586552763,42,14.40,46.47,71.37,3.31,
8,Tura,0,IN,1586552795,36,25.52,90.22,71.89,3.87,
9,Pozo Colorado,0,PY,1586552825,36,-23.49,-58.80,77.00,4.09,


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
lats=hotel_df["Latitude"]
long=hotel_df["Longitude"]
hotels=[]
params={
    "radius":50000,
    "types": "hotel",
    "keyword": "hotels",
    "key":g_key}
for index, row, in hotel_df.iterrows():
    lat=row["Latitude"]
    long=row["Longitude"]
    params["location"]=f"{lat},{long}"
    
    try:
        response=requests.get(base_url, params=params).json()
        
        hotel=response["results"][0]["name"]
        hotels.append(hotel)
    except (KeyError, IndexError):        
        hotel="NOT FOUND"
        hotels.append(hotel)
hotel_df["Hotel Name"]=hotels
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Maximum Temp.,Wind Speed,Hotel Name
0,Santiago,0,CL,1586552272,49,-33.46,-70.65,71.73,5.82,"The Ritz-Carlton, Santiago"
1,Gossaigaon,0,IN,1586552328,40,26.45,89.99,72.43,5.14,Resort Jayanti Hills Jungle Camp
2,Gwadar,0,PK,1586552465,68,25.12,62.33,79.54,3.62,Sadaf Resort
3,Pisco,0,PE,1586552460,30,-13.70,-76.22,73.11,9.17,"Hotel Paracas, a Luxury Collection Resort, Par..."
4,Benjamín Hill,0,MX,1586552496,18,30.17,-111.10,77.07,8.99,Hotel San José
5,Port Hedland,0,AU,1586552707,88,-20.32,118.57,71.60,5.82,The Esplanade Hotel
6,Chodavaram,0,IN,1586552740,63,17.83,82.95,79.48,4.00,Novotel Visakhapatnam Varun Beach
7,Marzuq,0,YE,1586552763,42,14.40,46.47,71.37,3.31,فندق الخـليـج
8,Tura,0,IN,1586552795,36,25.52,90.22,71.89,3.87,Hotel Polo Orchid
9,Pozo Colorado,0,PY,1586552825,36,-23.49,-58.80,77.00,4.09,Hotel Pozo Colorado


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display Map
fig


Figure(layout=FigureLayout(height='420px'))